In [1]:

# Special javascript code, that creates new BUTTON, which shows/hides all code.
# Button is visible also here in Python notebook, but really works and operates only in HTML export.

from IPython.display import HTML

HTML(
    """
<script>
    code_show = true;

    function code_toggle() {
        if (code_show) {
            document.querySelectorAll('.jp-CodeCell .jp-InputArea').forEach(function(el) {
               el.style.display = 'none';
            });
        } else {
            document.querySelectorAll('.jp-CodeCell .jp-InputArea').forEach(function(el) {
               el.style.display = 'block';
            });
        }
        code_show = !code_show
    }
</script>
<form action="javascript:code_toggle()">
    <input type="submit" value="Show / Hide code">
</form>
"""
)


In [2]:

import pandas as pd
import requests
import json
import ipywidgets as widgets
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from datetime import datetime, timedelta
from functools import reduce
from ipyaggrid import Grid
from IPython.display import display, HTML
from tzlocal import get_localzone

import warnings
warnings.filterwarnings("ignore")

# https://matplotlib.org/ipympl/#basic-example for more information
%matplotlib ipympl

# Set the default style for seaborn plots
sns.set_theme(style="whitegrid")
plt.ioff()  # Turn off interactive mode for matplotlib to prevent plots from displaying
# Base URL for API requests
BASE_URL = "https://singularity.easierdata.info/api"

# ................
# Global variables
# ................
# Color map properties for the stacked bar chart by the state of the deal
STATE_COLOR_MAP = {
    "active": {"color": "#6a4c93", "legend_title": "Sealed"},
    "published": {"color": "#1982c4", "legend_title": "Published"},
    "completed": {"color": "#8ac926", "legend_title": "Completed"},
    "proposed": {"color": "#ffca3a", "legend_title": "Proposed"},
    "proposal_expired": {"color": "#ff595e", "legend_title": "Proposal Expired"},
    "paused": {"color": "#6a737d", "legend_title": "Paused"},
    "<nil>": {"color": "#000001", "legend_title": "Other"},
}

# Show the last X-number of scheduled deals in stacked bar chart. Default is 20
NUM_OF_SCHEDULED_DEALS = 20

# Table style properties for creating interactive data tables
STYLE_PROPERTIES = {
    "color": "black",
    "font-size": "20px",
    "background-color": "lightgray",
    "padding": "10px",
    "border": "2px solid black",
    "text-align": "center",
}

# Style properties for the infographic card widgets
INFOGRAPHIC_CARD_STYLE_PROPERTIES = {
    "color": "white",
    "padding": "10px",
    "border-radius": "10px",
    "text-align": "center",
    "width": "275px",
    "height": "125px"
}

# Figure Dictionary
# Manage figures created by storing the figure objects with easy access by a reference key.
# To recall and activate a figure, use the key to retrieve the figure object from the dictionary
# Example: `FIGURE_REF['My-cool-figure'].number` will activate the figure object
FIGURE_REF = {}

# Get the user's local timezone
LOCAL_TZ = get_localzone()


def make_post_request(endpoint, data=None):
    """Make a POST request to a specified endpoint."""
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json; charset=UTF-8",
    }

    try:
        response = requests.post(
            f"{BASE_URL}{endpoint}", headers=headers, data=json.dumps(data)
        )
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None


def make_get_request(endpoint):
    """Make a POST request to a specified endpoint."""
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json; charset=UTF-8",
    }
    try:
        response = requests.get(f"{BASE_URL}{endpoint}", headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None


def process_preparations():
    preparations_response = make_get_request("/preparation")
    if preparations_response:
        preparations = preparations_response  # Assuming this is a list of dicts
        all_pieces_data = []
        all_deals_data = []

        for prep in preparations:

            # Get the pieces for each preparation
            prep_id = prep["id"]  # Assuming each preparation has an 'id'
            pieces_response = make_get_request(f"/preparation/{prep_id}/piece")
            if pieces_response:
                # Extract the pieces from each item in the list and then loop through each pieces in the list and extend into a dictionary of lists
                just_the_pieces = [item["pieces"] for item in pieces_response]
                just_the_pieces = [
                    item for sublist in just_the_pieces for item in sublist
                ]

                # Add the piece_name to the pieces data
                just_the_pieces = [{**d, "name": prep["name"]} for d in just_the_pieces]
                all_pieces_data.extend(just_the_pieces)

            # Get the pieces in deals for each preparation
            # Create data payload to send into post request
            data = {"preparations": [f"{prep_id}"]}

            deal_status_response = make_post_request("/deal", data)
            if deal_status_response:
                # Add preparation id to each deal status record
                deal_status_response = [
                    {**d, "preparationId": prep_id} for d in deal_status_response
                ]

                # Add deal status data
                all_deals_data.extend(deal_status_response)
            else:
                pass
                # print(
                #     f"No deals have been made for prepartion id {prep_id} - {prep['name']}"
                # )

        # Assuming all_pieces_data is a list of dicts
        all_pieces_df = pd.DataFrame(all_pieces_data)
        all_deals_df = pd.DataFrame(all_deals_data)

        return all_pieces_df, all_deals_df


def create_html_widget(content, style_dict=None):
    if style_dict is None:
        style_dict = {}

    # Convert the dictionary to a CSS style string
    style_string = "; ".join([f"{key}: {value}" for key, value in style_dict.items()])

    html_template = f"""
    <div style='{style_string}'>
        {content}
    </div>
    """
    return widgets.HTML(value=html_template)


def create_preparation_count_plot(df: pd.DataFrame, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    plot_obj = sns.countplot(data=df, x="preparationId", ax=ax)
    plot_obj.set_title("Number of pieces in each preparation")
    plot_obj.set_xlabel("Preparation ID")
    plot_obj.set_ylabel("Piece Count")
    add_count_labels_to_bars(plot_obj)
    if ax is None:
        plt.close(fig) # Close the figure to prevent it from displaying
    return plot_obj


def create_deal_state_plot(df, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # df.drop_duplicates(subset=["pieceCid"])
    plot_obj = sns.countplot(
        data=df,
        x="state",
        hue="preparationId",
        palette="viridis",
        ax=ax
    )

    plot_obj.set_title("Number of pieces by deal state and preparation")
    plot_obj.set_xlabel("Deal State")
    plot_obj.set_ylabel("Piece Count")
    add_count_labels_to_bars(plot_obj)
    if ax is None:
        plt.close(fig) # Close the figure to prevent it from displaying
    return plot_obj


def create_stacked_bar_chart(
    input_df: pd.DataFrame, color_map: dict, num_of_scheduled_deals: int = 20, ax=None
):
    """
    Prepare a stacked bar chart based on the input DataFrame.

    Args:
        input_df (pandas.DataFrame): The input DataFrame containing the necessary columns:
            'pieceCid', 'scheduleId', and 'state'.
        color_map (dict): A dictionary mapping each state to its corresponding color for the chart.
        num_of_scheduled_deals (int, optional): The number of scheduled deals to include in the chart.
            Defaults to 20.
        ax (matplotlib.axes.Axes, optional): The axes object to plot on. If None, a new figure and axes
            object will be created.

    Returns:
        matplotlib.axes.Axes: The axes object representing the plotted stacked bar chart.
    """
    # Ensure that the input DataFrame contains the necessary columns
    if not {"pieceCid", "scheduleId", "state"}.issubset(input_df.columns):
        raise ValueError(
            "The input DataFrame must contain the 'pieceCid', 'scheduleId', and 'state' columns."
        )

    deals_count_by_schedule_id_state = (
        input_df.drop_duplicates(subset=["pieceCid"])
        .groupby(["scheduleId", "state"])
        .size()
        .reset_index()
        .rename(columns={0: "pieceCount"})
    )

    pivot_deals_count_by_schedule_id_state = (
        deals_count_by_schedule_id_state.pivot_table(
            index="scheduleId",
            columns="state",
            values="pieceCount",
            aggfunc="sum",
            fill_value=0,
        )
    )

    pivot_deals_count_by_schedule_id_state = (
        pivot_deals_count_by_schedule_id_state.reindex(
            columns=color_map.keys(), fill_value=0
        )
    )


    pivot_deals_count_by_schedule_id_state = (
        pivot_deals_count_by_schedule_id_state.tail(num_of_scheduled_deals)
    )

    # Create a new figure and axes object if ax is not provided
    if ax is None:
        fig, ax = plt.subplots()

    # Create a dictionary to store the color values for each state
    color_map_colors = {
        state: color_map[state]["color"] for state in pivot_deals_count_by_schedule_id_state.columns
    }

    # Plotting without setting the final plot settings
    plot_obj = pivot_deals_count_by_schedule_id_state.plot(
        kind="bar", stacked=True, color=color_map_colors, ax=ax
    )


    # Set the plot title and labels
    plot_obj.set_title(
    f"Status of the last {num_of_scheduled_deals} Scheduled Deals by Piece Count"
    )
    plot_obj.set_xlabel("Schedule ID")
    plot_obj.set_ylabel("Total Pieces in Deal")
    plot_obj.set_xticklabels(
        plot_obj.get_xticklabels(), rotation=45
    )
    plot_obj.legend(title="Deal State Status")
    
    # Rename the legend titles to the corresponding title names in the color map dictionary
    for text in plot_obj.get_legend().texts:
        domain_value = text.get_text()
        text.set_text(color_map[domain_value]["legend_title"])
    
    if ax is None:
        plt.close()  # Close the figure to prevent it from displaying if created within the function

    return plot_obj


def calculate_deal_schedule_rate(input_deals_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the deal schedule rate over the past 14 days.

    Parameters:
    - input_deals_df: pd.DataFrame
        The input dataframe containing deal information.

    Returns:
    - pd.DataFrame
        A dataframe with the deal schedule rate over the past 14 days.
        The dataframe includes the local date, the number of pieces processed per day,
        and the rolling average of the pieces processed.

    Example Usage:
    >>> calculate_deal_schedule_rate(input_deals_df)
    """
    # Get the current date in the users local timezone
    current_date = datetime.now(LOCAL_TZ)

    # Create a list of dates for the past 14 days
    past_x_days = 14
    date_list = [(current_date - timedelta(days=x)) for x in range(past_x_days)]
    # Convert the list of dates to a dataframe and convert the datetime to date format
    date_df = pd.DataFrame(date_list, columns=["local_date"])
    date_df["local_date"] = date_df["local_date"].dt.date

    # Convert the createdAt column to datetime and extract the local time and date
    input_deals_df["createdAt"] = pd.to_datetime(input_deals_df["createdAt"], utc=True)
    input_deals_df["local_time"] = input_deals_df["createdAt"].dt.tz_convert(LOCAL_TZ)
    input_deals_df["local_date"] = input_deals_df["local_time"].dt.date

    # Group the deals by date and count the number of pieces processed each day.
    pieces_processed_per_day = (
        input_deals_df.groupby("local_date").size().reset_index(name="pieces_processed")
    )
    # Add missing dates that have no pieces processed and replace NaN values with 0
    # Note: the range of dates is from the current date to the past 14 days
    pieces_processed_per_day = pd.merge(
        date_df, pieces_processed_per_day, on="local_date", how="left"
    )
    pieces_processed_per_day.fillna(0, inplace=True)

    # Sort the dataframe by date and calculate the trend line using a rolling average
    pieces_processed_per_day = pieces_processed_per_day.sort_values(by="local_date")
    pieces_processed_per_day["rolling_avg"] = (
        pieces_processed_per_day["pieces_processed"].rolling(window=7).mean()
    )

    # Fill NaN values with 0
    pieces_processed_per_day.fillna(0, inplace=True)

    return pieces_processed_per_day


def plot_deals_over_time(daily_metrics_df: pd.DataFrame, ax=None) -> plt.Axes:
    """
    Plots the number of deals processed over time and the rolling average.

    Parameters:
        daily_metrics_df (pd.DataFrame): DataFrame containing daily metrics data.
        ax (plt.Axes, optional): Axes object to plot the chart on. If not provided, a new figure and axes object will be created.

    Returns:
        plt.Axes: The Axes object containing the plotted chart.
    """

    # Create a new figure and axes object if ax is not provided
    if ax is None:
        fig, ax = plt.subplots()

    # Assuming daily_metrics_df["local_date"] contains datetime.date objects
    # daily_metrics_df["local_date"] = daily_metrics_df["local_date"].astype(str)

    # Create plot axes for the bar chart and trend line
    ax.bar(
        daily_metrics_df["local_date"],
        daily_metrics_df["pieces_processed"],
    )
    ax.plot(
        daily_metrics_df["local_date"],
        daily_metrics_df["rolling_avg"],
    )

    # Set the properties of the plot
    ax.lines[0].set_color("red")
    ax.set_title("Scheduled deals over the past 14 days")
    ax.legend(["7-day Rolling Average"], loc="upper left")
    # Set the x-axis major locator to show the date every 2 days
    ax.xaxis.set_major_locator(mdates.DayLocator(interval=2))
    # if ax is None:
        # plt.close()  # Close the figure to prevent it from displaying if created within the function

    return ax


def create_interactive_data_table(
    df,
    enable_sorting=True,
    enable_filter=True,
    columns_fit="size_to_fit",
    quick_filter=True,
    show_toggle_edit=True,
    export_mode="buttons",
    export_csv=True,
    export_excel=True,
    export_df=True,
    theme_selection="ag-theme-fresh",
    grid_height=775,
):
    """
    Create a data table with customizable grid options and export features.

    Parameters:
    - df (pd.DataFrame): The dataframe to display in the table.
    - enable_sorting (bool): Enable or disable sorting.
    - enable_filter (bool): Enable or disable filtering.
    - columns_fit (str): How columns should fit in the table. Default is "size_to_fit".
    - quick_filter (bool): Enable or disable quick filtering.
    - show_toggle_edit (bool): Show or hide the toggle edit option.
    - export_mode (str): Set the mode of export buttons. Default is "buttons".
    - export_csv (bool): Enable or disable CSV export.
    - export_excel (bool): Enable or disable Excel export.

    Returns:
    - Grid: A table object with the specified options and data.
    """
    grid_options = {
        "columnDefs": [{"field": c, "autoHeaderHeight": True, "wrapHeaderText": True,"unSortIcon": False, "sortable": True} for c in df.columns],
        "enableSorting": enable_sorting,
        "enableFilter": enable_filter,
        "rowSelection": "multiple",
        # 'suppressColumnVirtualisation':True,
    }
    table = Grid(
        grid_data=df,
        grid_options=grid_options,
        columns_fit=columns_fit,
        quick_filter=quick_filter,
        show_toggle_edit=show_toggle_edit,
        export_mode=export_mode,
        export_csv=export_csv,
        export_excel=export_excel,
        export_to_df=export_df,
        theme=theme_selection,
        height=grid_height
        )
    return table


def generate_deal_summary_from_preparations(
    preparation_pieces: pd.DataFrame, deal_pieces: pd.DataFrame
) -> pd.DataFrame:
    """
    Calculate summary statistics by preparation from pieces and deals dataframes.

    Parameters:
    - preparation_pieces (pd.DataFrame): DataFrame containing all pieces information.
    - deal_pieces (pd.DataFrame): DataFrame containing all deals information.

    Returns:
    - pd.DataFrame: Summary statistics by preparation.
    """

    # Helper function to calculate total pieces by preparation
    def total_pieces_by_preparation(df):
        return (
            df.groupby("preparationId")
            .size()
            .reset_index()
            .rename(columns={0: "Total Pieces"})
        )

    # Helper function to calculate unique pieces in deals
    def unique_pieces_in_deals(df):
        return (
            df.drop_duplicates(subset=["pieceCid"])
            .groupby("preparationId")
            .size()
            .reset_index()
            .rename(columns={0: "Total Pieces that have been scheduled"})
        )

    def pieces_by_state(df, states, include=True, column_name="Number of pieces"):
        """
        Calculate pieces by state, allowing for filtering by multiple states.

        Parameters:
        - df (pd.DataFrame): DataFrame to operate on.
        - states (list of str): List of state values to filter by.
        - include (bool): If True, include the states in the filter; if False, exclude them.

        Returns:
        - pd.DataFrame: DataFrame grouped by preparationId with counts of pieces.
        """

        if isinstance(states, str):
            states = [states]

        if include:
            filtered_df = df[df["state"].isin(states)]
        else:
            filtered_df = df[~df["state"].isin(states)]

        return (
            filtered_df.drop_duplicates(subset=["pieceCid"])
            .groupby("preparationId")
            .size()
            .reset_index()
            .rename(columns={0: column_name})
        )

    # Calculate total pieces by preparation
    pieces_by_preparation = total_pieces_by_preparation(preparation_pieces)

    # Calculate unique pieces in deals
    unique_pieces = unique_pieces_in_deals(deal_pieces)

    # Calculate remaining pieces to be scheduled
    overall_total_pieces = pd.merge(
        pieces_by_preparation, unique_pieces, on="preparationId"
    )
    overall_total_pieces["Remaining number of pieces to Schedule"] = (
        overall_total_pieces["Total Pieces"]
        - overall_total_pieces["Total Pieces that have been scheduled"]
    )

    # Calculate pieces by specific states
    sealed_pieces = pieces_by_state(
        deal_pieces, "active", column_name="Number of sealed pieces"
    )
    expired_pieces = pieces_by_state(
        deal_pieces,
        "proposal_expired",
        column_name="Number of pieces with expired deals",
    )
    ongoing_pieces = pieces_by_state(
        all_deals_df,
        ["active", "proposal_expired"],
        include=False,
        column_name="Number of pieces in progress",
    )

    # Get the names of the preparations
    preparation_names = (
        all_pieces_df[["preparationId", "name"]]
        .drop_duplicates()
        .rename(columns={"name": "Preparation Name"})
    )

    # Merge all calculated statistics
    dfs_to_merge = [
        preparation_names,
        overall_total_pieces,
        sealed_pieces,
        expired_pieces,
        ongoing_pieces,
    ]
    summary_statistics = reduce(
        lambda left, right: pd.merge(left, right, on="preparationId", how="left"),
        dfs_to_merge,
    )

    # Fill NaN values with 0 and convert to integer
    summary_statistics.fillna(0, inplace=True)
    summary_statistics.style.format(precision=0)

    return summary_statistics


# Function to create the infographic cards
def generate_infographic_widgets(all_pieces_df, all_deals_df, summary_statistics_by_preparation):

    # Get a subset of all_deals_df with only the active deals
    active_deals_only = all_deals_df[all_deals_df["state"] == "active"]
    active_deals_only = active_deals_only.drop_duplicates(subset=["pieceCid"])

    # Calculate the total size of sealed content in GiB
    total_size_sealed_tib = calculate_total_size(active_deals_only)

    # Calculate the total size of content to seal in GiB
    total_size_to_seal_tib = calculate_total_size(all_pieces_df)

    # Calculate the total left to seal in GiB
    total_left_to_seal_tib = total_size_to_seal_tib - total_size_sealed_tib

    # Create a card for the total number of pieces in all preparations
    total_preparations_pieces_card = create_html_widget(
        content=f"<h2>Total pieces across all preparations</h2><p><h1><b>{summary_statistics_by_preparation['Total Pieces'].sum():,.0f}</h1></b></p>",
        style_dict={**INFOGRAPHIC_CARD_STYLE_PROPERTIES, "background-color": "#6ee2ef"},
    )

    # Create a card for the total number of sealed pieces
    sealed_pieces_card = create_html_widget(
        content=f"<h2>Total pieces sealed</h2><p><h1><b>{summary_statistics_by_preparation['Number of sealed pieces'].sum():,.0f}</h1></b></p>",
        style_dict={**INFOGRAPHIC_CARD_STYLE_PROPERTIES, "background-color": "#6ee2ef"},
    )

    # Create a card for the total number of sealed pieces left to be scheduled
    sealed_left_to_schedule_card = create_html_widget(
        content=f"<h2>Remaining pieces to be sealed</h2><p><h1><b>{summary_statistics_by_preparation['Remaining number of pieces to Schedule'].sum():,.0f}</h1></b></p>",
        style_dict={**INFOGRAPHIC_CARD_STYLE_PROPERTIES, "background-color": "#6ee2ef"},
    )

    # Create a card for the total size of content to seal in TiB
    size_to_seal_gib_card = create_html_widget(
        content=f"<h2>Total size of preparations</h2><p><h1><b>{total_size_to_seal_tib:,.0f} (TiB)</h1></b></p>",
        style_dict={**INFOGRAPHIC_CARD_STYLE_PROPERTIES, "background-color": "#162d30"},
    )

    # Create a card for the total size of sealed content in TiB
    size_sealed_gib_card = create_html_widget(
        content=f"<h2>Amount that has sealed</h2><p><h1><b>{total_size_sealed_tib:,.0f} (TiB)</h1></b></p>",
        style_dict={**INFOGRAPHIC_CARD_STYLE_PROPERTIES, "background-color": "#162d30"},
    )

    # Create a card for the total left to seal in TiB
    size_left_to_seal_gib_card = create_html_widget(
        content=f"<h2>Amount left to be sealed</h2><p><h1><b>{total_left_to_seal_tib:,.0f} (TiB)</h1></b></p>",
        style_dict={**INFOGRAPHIC_CARD_STYLE_PROPERTIES, "background-color": "#162d30"},
    )


    # Create a card for the total number of scheduled deals with errors
    scheduled_deals_in_progress_card = create_html_widget(
        content=f"<h2>Deals currently in progress</h2><p><h1><b>{summary_statistics_by_preparation['Number of pieces in progress'].sum():,.0f}</h1></b></p>",
        style_dict={**INFOGRAPHIC_CARD_STYLE_PROPERTIES, "background-color": "#33696f"},
    )

    # Create a card for the total number of expired deals
    expired_deals_card = create_html_widget(
        content=f"<h2>Deals that have expired</h2><p><h1><b>{summary_statistics_by_preparation['Number of pieces with expired deals'].sum():,.0f}</h1></b></p>",
        style_dict={**INFOGRAPHIC_CARD_STYLE_PROPERTIES, "background-color": "#33696f"},
    )

    # Create a grid layout to store the infographic cards
    infographic_card_grid = widgets.GridspecLayout(3, 4)

    # Center align the grid layout of the infographic cards so that they are centered on the page and not aligned to the left
    infographic_card_grid.layout = widgets.Layout(
        width="100%",
        grid_template_columns="repeat(4, 1fr)",
        grid_template_rows="repeat(3, 1fr)",
        justify_content="center",
    )

    # Add the infographic cards to the grid layout (Row by Col)
    infographic_card_grid[0, 0] = total_preparations_pieces_card
    infographic_card_grid[1, 0] = sealed_pieces_card
    infographic_card_grid[2, 0] = sealed_left_to_schedule_card

    infographic_card_grid[0, 1] = size_to_seal_gib_card
    infographic_card_grid[1, 1] = size_sealed_gib_card
    infographic_card_grid[2, 1] = size_left_to_seal_gib_card

    infographic_card_grid[1, 2] = scheduled_deals_in_progress_card
    infographic_card_grid[2, 2] = expired_deals_card

    return infographic_card_grid


def genereate_deal_metric_infographic_widgets(last_14_day_metric_df: pd.DataFrame):

    # Calculate the total number of pieces processed in the past 7 days
    processed_today = last_14_day_metric_df["pieces_processed"].iloc[-1]
    processed_last_7_days = last_14_day_metric_df["pieces_processed"].tail(7).sum()
    processed_last_14_days = last_14_day_metric_df["pieces_processed"].tail(14).sum()

    # Create text that displays date from 7 days ago
    date_7_days_ago = (
        last_14_day_metric_df["local_date"].iloc[-7].strftime("%m/%d/%y")
    )

    # Create text that displays the date from 14 days ago
    date_14_days_ago = (
        last_14_day_metric_df["local_date"].iloc[-14].strftime("%m/%d/%y")
    )

    # Create the infographic card
    total_pieces_processed_card = create_html_widget(
        content=f"<h2>Deals processed today</h2><p><h1><b>{processed_today:,.0f}</h1></b></p>",
        style_dict={
            **INFOGRAPHIC_CARD_STYLE_PROPERTIES,
            "background-color": "#51a6af",
        },
    )

    # Create the infographic card
    total_pieces_processed_last_7_days_card = create_html_widget(
        content=f"<h2>Deals processed since {date_7_days_ago}</h2><p><h1><b>{processed_last_7_days:,.0f}</h1></b></p>",
        style_dict={
            **INFOGRAPHIC_CARD_STYLE_PROPERTIES,
            "background-color": "#51a6af",
        },
    )

    # Create the infographic card
    total_pieces_processed_last_14_days_card = create_html_widget(
        content=f"<h2>Deals processed since {date_14_days_ago}</h2><p><h1><b>{processed_last_14_days:,.0f}</h1></b></p>",
        style_dict={
            **INFOGRAPHIC_CARD_STYLE_PROPERTIES,
            "background-color": "#51a6af",
        },
    )

    return (
        total_pieces_processed_card,
        total_pieces_processed_last_7_days_card,
        total_pieces_processed_last_14_days_card,
    )


def find_pieces_to_rechedule(all_deals_df: pd.DataFrame) -> list[str]:
    """
    Find pieces that need to be rescheduled based on the deal state.

    Parameters:
    - all_deals_df: A DataFrame containing information about the deals.

    Returns:
    - A list of pieceCids that need to be rescheduled.
    """

    ## Identifying pieceCids that need to be rescheduled
    # Group the all deals dataframe by pieceCid and count the number of times each pieceCid appears in the deals dataframe.
    pieceCid_counts = (
        all_deals_df.groupby("pieceCid").size().reset_index(name="deal_count")
    )

    # Filter out pieceCids that have a count of 1 or less
    pieceCid_counts = pieceCid_counts[pieceCid_counts["deal_count"] > 1]

    # Merge the pieceCid_counts dataframe with the all_deals_df dataframe to get the deal status for each pieceCid
    piece_cids_with_multiple_jobs = pd.merge(
        all_deals_df, pieceCid_counts, on="pieceCid", how="inner"
    )
    # sort the merged dataframe by pieceCid and state
    piece_cids_with_multiple_jobs = piece_cids_with_multiple_jobs.sort_values(
        by=["pieceCid", "state"]
    )

    # I need to find all the duplicate pieceCids that are only `proposal_expired` and return a the pieceCid and the number of times it appears in the
    # deals dataframe, group by pieceCid and count the number of times each pieceCid appears in the deals dataframe
    expired_deals_only = all_deals_df[all_deals_df["state"] == "proposal_expired"]
    expired_deals_only = (
        expired_deals_only.groupby("pieceCid").size().reset_index(name="deal_count")
    )

    piececid_expired_multiple_deals = expired_deals_only[
        expired_deals_only["deal_count"] > 1
    ]
    # Next I would like to identify which jobs contain the pieceCids that have multiple expired deals
    sched_jobs_with_multiple_expired_deals = all_deals_df[
        all_deals_df["pieceCid"].isin(piececid_expired_multiple_deals["pieceCid"])
    ]
    sched_jobs_with_multiple_expired_deals.sort_values(by=["pieceCid", "state"])

    # Next I need to confirm if the number of duplicate pieceCids in piececid_expired_multiple_deals matches the number of times the pieceCid appears in the deals dataframe for only the pieceCids that have multiple expired deals.
    assert (
        sched_jobs_with_multiple_expired_deals.shape[0]
        == piececid_expired_multiple_deals["deal_count"].sum()
    )

    ### Checking for the number of pieceCids that are expired but already have been sealed
    piececid_expired_single_deal = expired_deals_only[
        expired_deals_only["deal_count"] == 1
    ]

    # Check if there are any pieceCids from piececid_expired_single_deal that that have a pieceCide that also has a state of 'active' in piece_cids_with_multiple_jobs
    expired_subset_with_active_deals = piece_cids_with_multiple_jobs[
        piece_cids_with_multiple_jobs["state"] == "active"
    ]

    expired_subset_with_active_deals = piece_cids_with_multiple_jobs[
        piece_cids_with_multiple_jobs["state"] == "active"
    ]
    expired_subset_with_active_deals = expired_subset_with_active_deals[
        ["pieceCid", "state"]
    ]

    # Identify pieceCids in piececid_expired_single_deal that are not in expired_subset_with_active_deals
    piececid_expired_single_deal_not_active = piececid_expired_single_deal[
        ~piececid_expired_single_deal["pieceCid"].isin(
            expired_subset_with_active_deals["pieceCid"]
        )
    ]
    if len(piececid_expired_single_deal_not_active) > 0:
        print(
            f"pieceCids with a single expired deal that are not in active deals: {piececid_expired_single_deal_not_active.shape[0]}"
        )
        print(
            f"Here's the list of pieceCids that have only one expired deal and are not in active deals:"
        )
        print(piececid_expired_single_deal_not_active["pieceCid"].values)
    # else:
    #     print("All pieceCids with a single expired deal are also in active deals")

    ### Checking for the number of pieceCids that are that expired and have not been sealed
    piececid_expired_multiple_deals = expired_deals_only[
        expired_deals_only["deal_count"] > 1
    ]

    # Next I would like to identify which jobs contain the pieceCids that have multiple expired deals across all the scheduled deals
    # This will result with only rows that have been identified as having multiple expired deals and no sealed deals
    sched_jobs_with_multiple_expired_deals = all_deals_df[
        all_deals_df["pieceCid"].isin(piececid_expired_multiple_deals["pieceCid"])
    ]
    # Sort the dataframe by pieceCid and state
    sched_jobs_with_multiple_expired_deals.sort_values(by=["pieceCid", "state"])

    # Assert that piececid_expired_multiple_deals does not contain any pieceCids that have a state of 'active'
    assert (
        len(
            set(piececid_expired_multiple_deals["pieceCid"].values).intersection(
                set(expired_subset_with_active_deals["pieceCid"].values)
            )
        )
        == 0
    )

    # Assert that the number of rows in sched_jobs_with_multiple_expired_deals is equal to the sum of the `deal_count` in piececid_expired_multiple_deals
    assert (
        sched_jobs_with_multiple_expired_deals.shape[0]
        == piececid_expired_multiple_deals["deal_count"].sum()
    )

    return list(piececid_expired_multiple_deals["pieceCid"].values)


def calculate_total_size(df, return_in_tib=True):
    """
    Calculate the total size of content in GiB or TiB from a DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame containing the 'piece' column with size in KiB.
        return_in_tib (bool, optional): Whether to return the content size in TiB. Defaults to False.

    Returns:
        float: The total size of content in GiB or TiB.
    """
    total_size_bytes = df["pieceSize"].sum()
    total_size_gib = total_size_bytes / (1024**3)

    if return_in_tib:
        total_size_tib = total_size_gib / 1024
        return total_size_tib
    else:
        return total_size_gib


def add_count_labels_to_bars(ax):
    for p in ax.patches:
        if p.get_height() > 0:
            ax.annotate(
                f"{p.get_height():.0f}",
                (p.get_x() + p.get_width() / 2.0, p.get_height()),
                ha="center",
                va="bottom",
                color="black",
                size=12,
            )


# Helper function to convert Figure to bytes
def fig_to_bytes(fig):
    import io

    buf = io.BytesIO()
    fig.savefig(buf, format="png")
    plt.close(fig)  # Close the figure to prevent it from displaying
    buf.seek(0)
    return buf.read()


In [3]:

all_pieces_df, all_deals_df = process_preparations()

# ...............................................................................................................
# CREATE INTERACTIVE TABLES
# ...............................................................................................................
pieces_table = create_interactive_data_table(all_pieces_df)
deals_table = create_interactive_data_table(all_deals_df)
# Create a label widget for the tables
piece_label = create_html_widget(
    content="Preparation Pieces", style_dict=STYLE_PROPERTIES
)
deal_label = create_html_widget(
    content="Scheduled Deal Pieces", style_dict=STYLE_PROPERTIES
)

# Find pieces that haven't been sealed and need to be rescheduled due to expired deals and create an interactive table that only displays the pieces that need to be rescheduled
pieces_to_reschedule = find_pieces_to_rechedule(all_deals_df)
pieces_to_reschedule_table = create_interactive_data_table(
    all_pieces_df[all_pieces_df["pieceCid"].isin(pieces_to_reschedule)]
)

# ...............................................................................................................
# SUMMARY STATISTICS BY PREPARATION
# ...............................................................................................................

summary_statistics_by_preparation = generate_deal_summary_from_preparations(
    all_pieces_df, all_deals_df
)

summary_statistics_table = create_interactive_data_table(
    summary_statistics_by_preparation, grid_height=200
)

# ...............................................................................................................
# CREATE FIGURE LAYOUT TO STORE PLOTS
# ...............................................................................................................

# Create a new figure with a gridspec layout
FIGURE_REF["Figure-group-1"] = plt.figure(figsize=(15, 10), layout="tight")
fig = FIGURE_REF["Figure-group-1"]
gs = gridspec.GridSpec(2, 1)

# Top row for the two plots side by side in a 40:60 ratio
top_row = gridspec.GridSpecFromSubplotSpec(
    1, 2, subplot_spec=gs[0], width_ratios=[4, 6]
)
ax1 = fig.add_subplot(top_row[0, 0])
ax2 = fig.add_subplot(top_row[0, 1])

# Bottom row for the full-width plot
ax3 = fig.add_subplot(gs[1])

# deal metric plot
if "Figure-deal-metric-plot" not in FIGURE_REF:
    FIGURE_REF["Figure-deal-metric-plot"] = plt.figure(figsize=(15, 5), layout="tight")
    FIGURE_REF["Figure-deal-metric-plot"].add_subplot()

else:
    fig_deal_metric = FIGURE_REF["Figure-deal-metric-plot"]
    # Get the first (and only) Axes object from the list of axes in the figure
    deal_metric_ax_obj = fig_deal_metric.get_axes()[0]  # Access the first Axes object
    deal_metric_ax_obj.clear()  # Clear the axes
    # No need to add a new subplot if you're clearing the existing one

fig_deal_metric = FIGURE_REF["Figure-deal-metric-plot"]
deal_metric_ax_obj = fig_deal_metric.get_axes()[0]

# # ...............................................................................................................
# # COUNT PLOTS for Preparation ID and Deal State
# # ...............................................................................................................
preparation_count_plot = create_preparation_count_plot(all_pieces_df, ax=ax1)
deal_state_plot = create_deal_state_plot(all_deals_df, ax=ax2)

# ...............................................................................................................
# STACKED BAR CHART for Schedule ID and State
# ...............................................................................................................
stacked_bar_chart_plot = create_stacked_bar_chart(
    all_deals_df, STATE_COLOR_MAP, NUM_OF_SCHEDULED_DEALS, ax=ax3
)

# ...............................................................................................................

# DEAL SCHEDULE RATE PLOT
# ...............................................................................................................
deal_rate_df = calculate_deal_schedule_rate(all_deals_df)
plot_deals_over_time(deal_rate_df, ax=deal_metric_ax_obj)

# ...............................................................................................................
#
# ORGANIZE CONTENT AND DISPLAY
#
# ...............................................................................................................

# Store the tables in a tab widget
# Create a tab widget to store the interactive tables
interactive_tables = widgets.Tab()

if len(pieces_to_reschedule) == 0:
    reschedule_pieces_widget = create_html_widget(
        content="<h1>All previously expired pieces have already been sealed</h1>",
        style_dict={**STYLE_PROPERTIES, "background-color": "lightgray"},
    )

else:
    # Create an HTML widget to display the pieces that need to be rescheduled
    reschedule_pieces_title = create_html_widget(
        content=f"{len(pieces_to_reschedule)} pieces have been identified that need to be rescheduled",
        style_dict={**STYLE_PROPERTIES, "background-color": "lightgray"},
    )
    # Create a string of text that will be passed into the HTML widget to display the list of pieces that need to be rescheduled
    widget_text = f"<p><h3>The following pieces have multiple expired deals that have yet to be sealed:</h3><br> {', '.join(pieces_to_reschedule)}</p>"
    # create HTML widget to display the list of pieces that need to be rescheduled. Create an html widget that best displays the list of pieces that need to be rescheduled
    reschedule_pieces_list_widget = widgets.HTML(
        value=widget_text,
        layout=widgets.Layout(
            color="blue",
            font_size="20px",
            background_color="lightgray",
            padding="10px",
            border="2px solid black",
            text_align="center",
        ),
    )
    # Create a VBox to store the reschedule pieces widgets
    reschedule_pieces_widget = widgets.VBox(
        children=[
            reschedule_pieces_title,
            reschedule_pieces_list_widget,
            pieces_to_reschedule_table,
        ],
        layout=widgets.Layout(height="auto", width="auto"),
    )

interactive_tables.children = [pieces_table, deals_table, reschedule_pieces_widget]

# Set the tab titles
tab_titles = ["Preparation Pieces", "Deal Pieces", "Pieces to Reschedule"]
for i, title in enumerate(tab_titles):
    interactive_tables.set_title(i, title)

# Create a tab widget to store the summary statistics
# Create infographic cards widget
infographic_overview_card_widgets = generate_infographic_widgets(
    all_pieces_df, all_deals_df, summary_statistics_by_preparation
)

# Generate the deal metric infographic widgets
today, thisweek, previousweek = genereate_deal_metric_infographic_widgets(deal_rate_df)

# Generate rolling average of deals processed in the past 7 days
latest_rolling_avg = deal_rate_df.iloc[-1].rolling_avg
latest_rolling_avg_widget = create_html_widget(
    content=f"<h2>Avg number of pieces processed daily</h2><p><h1><b>{latest_rolling_avg:,.0f}</h1></b></p>",
    style_dict={**INFOGRAPHIC_CARD_STYLE_PROPERTIES, "background-color": "#33696f"},
)

# Add the deal metric infographic widgets to the VBox at specific grid locations
infographic_overview_card_widgets[0, 2] = latest_rolling_avg_widget
infographic_overview_card_widgets[0, 3] = today
infographic_overview_card_widgets[1, 3] = thisweek
infographic_overview_card_widgets[2, 3] = previousweek

# Create an HTML widget that acts as the header title for the infographic cards that will be displayed above the inforgraphic cards
Overview_title = create_html_widget(
    content="Overview of Scheduled Deals",
    style_dict={**STYLE_PROPERTIES, "background-color": "lightgray"},
)

# Create an HTML widget that acts as the header title for the infographic cards that will be displayed above the inforgraphic cards
prep_table_title = create_html_widget(
    content="Summary Statistics by Preparation",
    style_dict={**STYLE_PROPERTIES, "background-color": "lightgray"},
)

summary_statistics_tab = widgets.VBox(
    children=[
        Overview_title,
        infographic_overview_card_widgets,
        prep_table_title,
        summary_statistics_table,
    ],
    layout=widgets.Layout(height="auto", width="auto"),
)

# ...............................................................................................................
# Create a tab widget to store the plots
# ...............................................................................................................
default_layout_properties = widgets.Layout(
    align_items="center",
    object_fit="fill",
    width="auto",
)

# convert figures to image widgets
combined_figure_set_widget = widgets.Image(
    value=fig_to_bytes(FIGURE_REF["Figure-group-1"].get_figure()),
    format="png",
)

scheduled_rate_widget = widgets.Image(
    value=fig_to_bytes(FIGURE_REF["Figure-deal-metric-plot"].get_figure()),
    format="png",
)

# # Alternative method to display the plots using Output widget
# combined_figure_set_widget = widgets.Output(layout=default_layout_properties)
# with combined_figure_set_widget:
#     display(FIGURE_REF["Figure-group-1"].get_figure()
plots_tab = widgets.VBox(
    children=[combined_figure_set_widget, scheduled_rate_widget],
    layout=default_layout_properties,
)

# Create a tab widget to store all the content
all_content = widgets.Tab()

# Set the tab titles
tab_titles = ["Deal Breakdown", "Interactive Tables", "Plots"]

# Set the tab contents
all_content.children = [summary_statistics_tab, interactive_tables, plots_tab]

# Set the tab titles
for i, title in enumerate(tab_titles):

    all_content.set_title(i, title)

# Display the tab widget
display(all_content)
